<a href="https://colab.research.google.com/github/aayushj9/LLM_finetuning/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
# STEP 1: Install Required Libraries
!pip install -q transformers datasets evaluate

In [27]:
# STEP 2: Imports
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer
from datasets import Dataset
import evaluate
import torch

In [35]:
# STEP 3: Expanded Sample Airline Chatbot Dataset
data = [
    {"instruction": "Book a flight", "input": "I want to book a flight from Delhi to Dubai on 15 July", "output": "Sure! Booking a flight from Delhi to Dubai on 15 July. Please confirm your time preference."},
    {"instruction": "Flight status", "input": "What is the status of flight AI302?", "output": "Flight AI302 is currently on time and scheduled to depart at 10:30 AM."},
    {"instruction": "Baggage policy", "input": "What's the baggage allowance for international flights?", "output": "You are allowed 2 checked bags (23kg each) and 1 cabin bag (7kg) on international flights."},
    {"instruction": "Cancel ticket", "input": "I need to cancel my ticket for flight AI203", "output": "Your ticket for flight AI203 has been canceled. A refund will be processed within 5-7 business days."},
    {"instruction": "Reschedule flight", "input": "Can I reschedule my flight from Mumbai to London?", "output": "Yes, you can reschedule your flight. Please share the new preferred date."},
    {"instruction": "Flight delay", "input": "Is flight AI405 delayed?", "output": "Flight AI405 is delayed by 2 hours due to bad weather."},
    {"instruction": "Meal preference", "input": "Can I change my meal to vegetarian?", "output": "Your meal preference has been updated to vegetarian."},
    {"instruction": "Seat selection", "input": "I want a window seat for my flight", "output": "Window seat has been reserved for your booking."},
    {"instruction": "Upgrade class", "input": "Can I upgrade to business class?", "output": "Yes, upgrade to business class is available for an additional fee."},
    {"instruction": "Luggage lost", "input": "I lost my luggage on flight AI129", "output": "We are sorry to hear that. A lost baggage claim has been initiated for flight AI129."},
    {"instruction": "Refund status", "input": "What is the status of my refund for ticket 78231?", "output": "Your refund for ticket 78231 has been processed and will reflect in 3-5 days."},
    {"instruction": "Check-in time", "input": "When should I check in for an international flight?", "output": "Check-in for international flights opens 3 hours before departure."},
    {"instruction": "Boarding gate", "input": "What is the gate for flight AI302?", "output": "Flight AI302 departs from gate 7A."},
    {"instruction": "Visa support", "input": "Do I need a visa to fly to the USA?", "output": "Yes, a valid US visa is required before travel."},
    {"instruction": "Infant travel", "input": "Can I carry baby food on board?", "output": "Yes, you may carry baby food in reasonable quantities."},
    {"instruction": "Flight change fee", "input": "Is there a fee to change my flight?", "output": "Yes, a change fee applies. It depends on your fare type."},
    {"instruction": "COVID policy", "input": "Do I need a COVID test for domestic travel?", "output": "No COVID test is required for domestic travel currently."},
    {"instruction": "Wi-Fi availability", "input": "Is there Wi-Fi on the flight?", "output": "Yes, Wi-Fi is available on select international flights."},
    {"instruction": "In-flight entertainment", "input": "What entertainment is available on board?", "output": "We offer movies, TV shows, music, and games on personal screens."},
    {"instruction": "Loyalty program", "input": "How can I earn miles?", "output": "You earn miles every time you fly with us or our partners."},
    {"instruction": "Special assistance", "input": "I need a wheelchair for airport transit", "output": "Wheelchair assistance has been requested for your booking."},
    {"instruction": "Pet travel", "input": "Can I bring my dog on the flight?", "output": "Yes, pets are allowed in cabin or cargo, based on size and regulations."},
    {"instruction": "Weather delay", "input": "Is flight AI209 affected by the storm?", "output": "Flight AI209 is delayed by 1 hour due to the ongoing storm."},
    {"instruction": "Connecting flight", "input": "How do I know my connecting flight gate?", "output": "Your connecting gate details will be available upon landing."},
    {"instruction": "Flight time", "input": "What time does flight AI701 leave?", "output": "Flight AI701 is scheduled to depart at 6:45 PM."},
    {"instruction": "Change passenger name", "input": "Can I correct the name on my ticket?", "output": "Name corrections are allowed for minor changes. Please contact support."},
    {"instruction": "Senior citizen discount", "input": "Is there a discount for seniors?", "output": "Yes, senior citizens are eligible for a 10% discount on base fare."},
    {"instruction": "Student fare", "input": "Do you offer student discounts?", "output": "Yes, student discounts are available with valid ID proof."},
    {"instruction": "Excess baggage", "input": "Can I carry 5kg extra baggage?", "output": "Yes, additional baggage is allowed with extra charges."},
    {"instruction": "Pre-book meal", "input": "Can I pre-book a meal?", "output": "Yes, you can pre-book your meal through the Manage Booking section."}
]


In [36]:
# STEP 4: Create Hugging Face Dataset
def preprocess(example):
    example["text"] = f"Instruction: {example['instruction']}\nInput: {example['input']}"
    example["label"] = example["output"]
    return example

dataset = Dataset.from_list(data).map(preprocess)

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

In [37]:
# STEP 5: Load Model and Tokenizer
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


In [38]:
# STEP 6: Tokenize Dataset
def tokenize_function(example):
    inputs = tokenizer(example["text"], padding="max_length", truncation=True, max_length=128)
    labels = tokenizer(example["label"], padding="max_length", truncation=True, max_length=128)
    inputs["labels"] = labels["input_ids"]
    return inputs

tokenized_dataset = dataset.map(tokenize_function, remove_columns=dataset.column_names)


Map:   0%|          | 0/30 [00:00<?, ? examples/s]

In [39]:
# STEP 7: Define Training Arguments and Trainer
training_args = TrainingArguments(
    output_dir="./flan-airline-bot",
    per_device_train_batch_size=2,
    num_train_epochs=5,
    logging_steps=1,
    save_total_limit=1,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer
)

<ipython-input-39-4d33a67736f6>:11: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [40]:
# STEP 8: Train the Model
trainer.train()


Step,Training Loss
1,39.235100
2,40.022900
3,38.631800
4,38.353000
5,38.915800
6,34.654700
7,33.903000
8,32.396500
9,37.958300
10,32.057000


TrainOutput(global_step=75, training_loss=23.3275696182251, metrics={'train_runtime': 387.6072, 'train_samples_per_second': 0.387, 'train_steps_per_second': 0.193, 'total_flos': 6970893926400.0, 'train_loss': 23.3275696182251, 'epoch': 5.0})

In [41]:
# Step 9  Evalaation
from transformers import pipeline

pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

correct = 0
for example in data:
    prompt = f"Instruction: {example['instruction']}\nInput: {example['input']}"
    prediction = pipe(prompt, max_new_tokens=50)[0]['generated_text']
    print("\n Instruction:", example["instruction"])
    print(" Input:", example["input"])
    print(" Expected:", example["output"])
    print(" Predicted:", prediction)

    if example["output"].split()[0].lower() in prediction.lower():  # simple keyword check
        correct += 1

accuracy = correct / len(data)
print(f"\n Simple Match Accuracy: {accuracy * 100:.2f}%")


Device set to use cpu



 Instruction: Book a flight
 Input: I want to book a flight from Delhi to Dubai on 15 July
 Expected: Sure! Booking a flight from Delhi to Dubai on 15 July. Please confirm your time preference.
 Predicted: i want to book a flight from Delhi to Dubai

 Instruction: Flight status
 Input: What is the status of flight AI302?
 Expected: Flight AI302 is currently on time and scheduled to depart at 10:30 AM.
 Predicted: Flight status

 Instruction: Baggage policy
 Input: What's the baggage allowance for international flights?
 Expected: You are allowed 2 checked bags (23kg each) and 1 cabin bag (7kg) on international flights.
 Predicted: ad & b & b & b & b & b & b & b & c & b & c & b & 

 Instruction: Cancel ticket
 Input: I need to cancel my ticket for flight AI203
 Expected: Your ticket for flight AI203 has been canceled. A refund will be processed within 5-7 business days.
 Predicted: I need to cancel my ticket

 Instruction: Reschedule flight
 Input: Can I reschedule my flight from Mumba